In [3]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime, timedelta
import altair as alt
from pandas.io.json import json_normalize
import requests
import yahoo_fin.stock_info as yf
from alpha_vantage.fundamentaldata import FundamentalData
import json
from fbprophet import Prophet 

D = 400
date_D_days_ago = datetime.now() - timedelta(days=D)
now = datetime.now()

start_date = date_D_days_ago.strftime('%F')
end_date = now.strftime('%F')

apikey='2HCDHLMKIK99309W'
tech_stocks = ['AAPL', 'PTON', 'TSLA','ADYEY']

portfolio2 = ['CSCO','PTON','AI','TSLA','ADYEY','UBER']

tech_stocks[0]


# Operating CashFlow minus Capital Expenditures


'AAPL'

In [ ]:
#Get Data from Yahoo
def StockData(ticker, start_d, end_d):
    data = web.get_data_yahoo(ticker, start = start_d, end = end_d)
    price =  pd.DataFrame(data['Adj Close'])
    
    #volume = pd.DataFrame(data['Volume'])
    return price

In [ ]:
def Forecast(df,ticker):
    df = df.fillna(0)
    tit = ''+ticker
    df = df.reset_index()
    df.columns = ['ds','y']
    prophet = Prophet()
    prophet.fit(df)
    future_prices=prophet.make_future_dataframe(periods=365)
    future = prophet.predict(future_prices)
    future = future[['ds','trend','yhat_lower','yhat_upper']]
    return future

In [ ]:
price = StockData(portfolio2, start_date, end_date)

In [ ]:
def Performance(p):
   
    perform = pd.DataFrame(0, index = p.columns, columns=['performance','trend','yhat_lower','yhat_upper'])
       
    df =(p.resample('W').ffill().pct_change() + 1).cumprod().fillna(0).tail(1)
    perform['performance'] = df.iloc[0,:].apply(pct) 
    
    for i in p.columns:
        f = Forecast(p[i],i)
        f.index = f['ds']
        f = f.drop(['ds'], axis=1)
        f = f[now:]
        print(f.tail(1))
        f = (f.resample('W').ffill().pct_change()+1).cumprod().fillna(0).tail(1).applymap(pct)
        print(f)
        perform.loc[i,'trend'] = f.iloc[0,0]
        perform.loc[i,'yhat_lower'] = f.iloc[0,1]
        perform.loc[i,'yhat_upper'] = f.iloc[0,2]
    return perform

In [ ]:
p = Performance(price)

In [ ]:
p

In [ ]:
p['yhat_upper'].astype(float) - p['yhat_lower'].astype(float)

In [ ]:
p

In [ ]:
t.index = t['ds']

In [ ]:
t = t.drop(['ds'], axis=1)

In [ ]:
t = t[now:]
dig = lambda x: '{:.2f}'.format(x)
pct = lambda x: '{:.2%}'.format(x)

In [ ]:
p2 = (t.resample('W').ffill().pct_change()+1).cumprod().fillna(0).tail(1).applymap(dig)

In [ ]:
p2


In [ ]:
df1 = pd.DataFrame({'key' : ['a','b','c'],'value': range(3)}, index = ['a1','x2','a3'])
df2 = pd.DataFrame({'key' : ['x','y','x','v'],'stuff': range(4)}, index = ['a1','a2','a3','a4'])

In [ ]:
df1

In [ ]:
df2

In [ ]:
df1.combine_first(df2)

In [ ]:
df1.index.str.match('a3').any()

In [ ]:
df1.columns.values[0]

In [6]:
win = yf.get_day_gainers()


In [15]:


win.describe


<bound method NDFrame.describe of    Symbol                                  Name  Price (Intraday)  Change  \
0   RLLCF              Rolls-Royce Holdings plc            0.0470  0.0301   
1     CAN                           Canaan Inc.           17.5700  4.5300   
2   DFIFF         Diamond Fields Resources Inc.            0.3121  0.0716   
3    FINV                     FinVolution Group            7.7100  1.5300   
4   CMCLF            China Molybdenum Co., Ltd.            0.8500  0.1438   
..    ...                                   ...               ...     ...   
95    CMA                 Comerica Incorporated           65.6100  3.3000   
96     PK            Park Hotels & Resorts Inc.           20.7400  1.0600   
97   COOP                 Mr. Cooper Group Inc.           31.3600  1.5200   
98    CMP  Compass Minerals International, Inc.           67.7100  3.5300   
99  REPYY                          Repsol, S.A.           11.4100  0.5700   

    % Change        Volume  Avg Vol (3 mo

In [8]:
lose = yf.get_day_losers()
lose = lose.sort_values(by='Market Cap',ascending = False).head(5).sort_values(by='% Change',ascending = True)

In [9]:
lose

,Symbol,Name,Price (Intraday),Change,% Change,Volume,Avg Vol (3 month),Market Cap,PE Ratio (TTM)
6,PLTR,Palantir Technologies Inc.,28.600,-3.310,-10.36,1.099040e+08,71909000.0,5.371800e+10,NaN
14,BKRKF,PT Bank Rakyat Indonesia (Persero) Tbk,0.328,-0.032,-8.89,2.141300e+14,1269000.0,3.735900e+10,18.22
83,TWLO,Twilio Inc.,414.450,-20.850,-4.79,1.499000e+06,2003000.0,6.651100e+10,NaN
96,NTES,"NetEase, Inc.",126.210,-5.910,-4.47,1.501000e+06,2533000.0,8.673700e+10,18.92
91,CVS,CVS Health Corporation,70.970,-3.240,-4.37,1.101100e+07,7516000.0,9.289300e+10,11.73


In [ ]:
active = yf.get_day_most_active()
active = lose.sort_values(by='Market Cap',ascending = False).head(5)

In [ ]:
active

In [ ]:
yf.get_quote_table("aapl")

In [16]:
yf.get_quote_table("aapl")["EPS (TTM)"]

3.69

In [ ]:
performance = (price.resample('W').ffill().pct_change() + 1).cumprod().fillna(0).tail(1)

In [ ]:
df = pd.DataFrame(0, index= price.columns, columns =['performance','EPS'])

In [ ]:
df['performance'] = performance.tail(1)

In [ ]:
df['performance'] = performance.iloc[0,:]

In [ ]:
my_EPS = lambda x: yf.get_quote_table(x)["EPS (TTM)"]

In [ ]:
for i in df.index:
    EPS = yf.get_quote_table(i)["EPS (TTM)"]
    df.loc[i,'EPS'] = EPS
    
    

In [ ]:
data

In [ ]:
def Plot_EPS(price):
    performance = (price.resample('W').ffill().pct_change() + 1).cumprod().fillna(0).tail(1)
    df = pd.DataFrame(0, index= price.columns, columns =['performance','EPS'])
    df['performance'] = performance.tail(1)
    df['performance'] = performance.iloc[0,:]

    for i in df.index:
        EPS = yf.get_quote_table(i)["EPS (TTM)"]
        df.loc[i,'EPS'] = EPS



    df = df.reset_index()

    pic = alt.Chart(df).mark_point().encode(alt.X('performance:Q',scale=alt.Scale(zero=False)
        ),y='EPS:Q', 
        color='Symbols:N', 
        size=alt.Size("performance:Q", scale=alt.Scale(range=[0, 1000])), 
        tooltip=['Symbols:N', 'performance:N','EPS:N'])
    return pic

In [ ]:
table = yf.get_quote_table('CSCO')

pd.read_html(table, index=0)

In [ ]:
table.values